In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference_df, analysis_df, _ = nml.load_synthetic_car_loan_dataset()
display(reference_df.head())

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


In [ ]:
print(reference_df.head().to_markdown(tablefmt="grid"))

+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+--------------

In [ ]:
non_feature_columns = ['timestamp', 'y_pred_proba', 'y_pred', 'repaid']

# Define feature columns
feature_column_names = [
    col for col in reference_df.columns
    if col not in non_feature_columns
]

calc = nml.DomainClassifierCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
calc.fit(reference_df)
results = calc.calculate(analysis_df)

In [ ]:
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                    classifier_auroc                  \
                 end_date    period            value upper_threshold   
0 2018-11-30 00:27:16.848  analysis         0.502704            0.65   
1 2018-12-30 07:03:16.848  analysis         0.496390            0.65   
2 2019-01-29 13:39:16.848  analysis         0.490815            0.65   
3 2019-02-28 20:15:16.848  analysis         0.493005            0.65   
4 2019-03-31 02:51:16.848  analysis         0.503402            0.65   
5 2019-04-30 09:27:16.848  analysis         0.913519            0.65   
6 2019-05-30 16:03:16.848  analysis         0.913364            0.65   
7 2019-06-29 22:39:16.848  analysis         0.916356            0.65   
8 2019-07-30 05:15:16.848  analysis         0.913297            0.65   
9 2019-08-29 11:51:16.848  analysis         0.916694            0.65   

                          
  lower_threshold  alert  
0            0.45  False  
1            0.45  False  
2            0.45  False  
3            0.45  False  
4            0.45  False  
5            0.45   True  
6            0.45   True  
7            0.45   True  
8            0.45   True  
9            0.45   True

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+----------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | classifier_auroc   |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | value              | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+============================+============+======================+=====================+=====================+===========+
| 0  | [0:4999]      | 0               | 0               | 4999          | 2018-10-30 18:00:00 | 2018-11-30 00:27:16.848000 | an

In [ ]:
display(results.filter(period='reference').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-01-01 00:00:00   
1    [5000:9999]           1        5000      9999 2018-01-31 06:36:00   
2  [10000:14999]           2       10000     14999 2018-03-02 13:12:00   
3  [15000:19999]           3       15000     19999 2018-04-01 19:48:00   
4  [20000:24999]           4       20000     24999 2018-05-02 02:24:00   
5  [25000:29999]           5       25000     29999 2018-06-01 09:00:00   
6  [30000:34999]           6       30000     34999 2018-07-01 15:36:00   
7  [35000:39999]           7       35000     39999 2018-07-31 22:12:00   
8  [40000:44999]           8       40000     44999 2018-08-31 04:48:00   
9  [45000:49999]           9       45000     49999 2018-09-30 11:24:00   

                                     classifier_auroc                  \
                 end_date     period            value upper_threshold   
0 2018-01-31 06:27:16.848  reference         0.508085            0.65   
1 2018-03-02 13:03:16.848  reference         0.505428            0.65   
2 2018-04-01 19:39:16.848  reference         0.506587            0.65   
3 2018-05-02 02:15:16.848  reference         0.499824            0.65   
4 2018-06-01 08:51:16.848  reference         0.507135            0.65   
5 2018-07-01 15:27:16.848  reference         0.498486            0.65   
6 2018-07-31 22:03:16.848  reference         0.501805            0.65   
7 2018-08-31 04:39:16.848  reference         0.494281            0.65   
8 2018-09-30 11:15:16.848  reference         0.505302            0.65   
9 2018-10-30 17:51:16.848  reference         0.502734            0.65   

                          
  lower_threshold  alert  
0            0.45  False  
1            0.45  False  
2            0.45  False  
3            0.45  False  
4            0.45  False  
5            0.45  False  
6            0.45  False  
7            0.45  False  
8            0.45  False  
9            0.45  False

In [ ]:
print_multi_index_markdown(results.filter(period='reference').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+----------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | classifier_auroc   |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | value              | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+============================+============+======================+=====================+=====================+===========+
| 0  | [0:4999]      | 0               | 0               | 4999          | 2018-01-01 00:00:00 | 2018-01-31 06:27:16.848000 | re

In [ ]:
figure = results.plot()
figure.show()

In [ ]:
figure.write_image('../_static/tutorials/detecting_data_drift/multivariate_drift_detection/classifier-for-drift-detection.svg')